<a href="https://colab.research.google.com/github/2905-mrin/ML_F_PES2UG23CS353_Mrinal/blob/main/ML_Week3_PES2UG23CS353_Mrinal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%writefile PES_F_PES2UG23CS353_Lab3.py


import numpy as np
import pandas as pd

def get_entropy_of_dataset(data):
    """
    Calculates the entropy of the entire dataset.
    Entropy is a measure of impurity or disorder in the data.
    """
    target_col = data[:, -1]

    _, counts = np.unique(target_col, return_counts=True)

    probabilities = counts / len(target_col)

    dataset_entropy = -np.sum(probabilities * np.log2(probabilities + 1e-9))

    return dataset_entropy


def get_avg_info_of_attribute(data, attribute):
    """
    Calculates the average information (weighted entropy) for a specific attribute.
    This tells us the average impurity remaining after splitting the data by this attribute.
    """
    attribute_col = data[:, attribute]

    unique_attribute_values = np.unique(attribute_col)

    avg_info = 0.0

    for value in unique_attribute_values:
        subset_data = data[data[:, attribute] == value]

        weight = len(subset_data) / len(data)

        avg_info += weight * get_entropy_of_dataset(subset_data)

    return avg_info


def get_information_gain(data, attribute):
    """
    Calculates the information gain for a given attribute.
    Information gain measures how much the entropy is reduced by splitting on this attribute.
    A higher information gain means a better feature for splitting.
    """
    total_entropy = get_entropy_of_dataset(data)

    avg_info_attribute = get_avg_info_of_attribute(data, attribute)

    information_gain = total_entropy - avg_info_attribute

    return information_gain


def get_selected_attribute(data):
    """
    Finds the best attribute to split on by selecting the one with the highest information gain.
    """
    num_attributes = data.shape[1] - 1
    information_gains = {}
    selected_attribute = -1
    max_gain = -1

    for i in range(num_attributes):
        gain = get_information_gain(data, i)
        information_gains[i] = gain
        if gain > max_gain:
            max_gain = gain
            selected_attribute = i

    if max_gain <= 0:
        selected_attribute = -1

    return information_gains, selected_attribute



# Decision Tree Definition

class Node:

    def __init__(self, feature_index=None, predicted_class=None):
        self.feature_index = feature_index
        self.children = {}
        self.predicted_class = predicted_class

    def is_leaf_node(self):
        return self.predicted_class is not None

class DecisionTree:

    def __init__(self):
        self.root = None

    def fit(self, data):

        self.root = self._build_tree(data)

    def _build_tree(self, data):

        target_col = data[:, -1]

        if len(np.unique(target_col)) == 1:
            return Node(predicted_class=target_col[0])

        if data.shape[1] == 1:
            unique_classes, counts = np.unique(target_col, return_counts=True)
            majority_class = unique_classes[np.argmax(counts)]
            return Node(predicted_class=majority_class)

        _, best_feature_index = get_selected_attribute(data)

        if best_feature_index == -1:
            unique_classes, counts = np.unique(target_col, return_counts=True)
            majority_class = unique_classes[np.argmax(counts)]
            return Node(predicted_class=majority_class)

        node = Node(feature_index=best_feature_index)

        feature_values = np.unique(data[:, best_feature_index])

        for value in feature_values:
            subset_data = data[data[:, best_feature_index] == value]
            subset_data_without_feature = np.delete(subset_data, best_feature_index, axis=1)

            subtree = self._build_tree(subset_data_without_feature)
            node.children[value] = subtree

        return node

    def predict(self, records):

        predictions = [self._traverse_tree(record, self.root) for record in records]
        return np.array(predictions)

    def _traverse_tree(self, record, node):

        if node.is_leaf_node():
            return node.predicted_class

        feature_value = record[node.feature_index]

        if feature_value in node.children:
            next_node = node.children[feature_value]
            next_record = np.delete(record, node.feature_index)
            return self._traverse_tree(next_record, next_node)
        else:
            return None

Writing PES_F_PES2UG23CS353_Lab3.py


In [ ]:
# To visualize the tree for the mushroom dataset
!python test.py --ID PES_F_PES2UG23CS353_Lab3 --data mushrooms.csv

Running tests with PYTORCH framework
 target column: 'class' (last column)
Original dataset info:
Shape: (8124, 23)
Columns: ['cap-shape', 'cap-surface', 'cap-color', 'bruises', 'odor', 'gill-attachment', 'gill-spacing', 'gill-size', 'gill-color', 'stalk-shape', 'stalk-root', 'stalk-surface-above-ring', 'stalk-surface-below-ring', 'stalk-color-above-ring', 'stalk-color-below-ring', 'veil-type', 'veil-color', 'ring-number', 'ring-type', 'spore-print-color', 'population', 'habitat', 'class']

First few rows:

cap-shape: ['x' 'b' 's' 'f' 'k'] -> [5 0 4 2 3]

cap-surface: ['s' 'y' 'f' 'g'] -> [2 3 0 1]

cap-color: ['n' 'y' 'w' 'g' 'e'] -> [4 9 8 3 2]

class: ['p' 'e'] -> [1 0]

Processed dataset shape: torch.Size([8124, 23])
Number of features: 22
Features: ['cap-shape', 'cap-surface', 'cap-color', 'bruises', 'odor', 'gill-attachment', 'gill-spacing', 'gill-size', 'gill-color', 'stalk-shape', 'stalk-root', 'stalk-surface-above-ring', 'stalk-surface-below-ring', 'stalk-color-above-ring', 's

In [ ]:
# Test with the Tic-Tac-Toe dataset
!python test.py --ID PES_F_PES2UG23CS353_Lab3 --data tictactoe.csv

Running tests with PYTORCH framework
 target column: 'Class' (last column)
Original dataset info:
Shape: (958, 10)
Columns: ['top-left-square', 'top-middle-square', 'top-right-square', 'middle-left-square', 'middle-middle-square', 'middle-right-square', 'bottom-left-square', 'bottom-middle-square', 'bottom-right-square', 'Class']

First few rows:

top-left-square: ['x' 'o' 'b'] -> [2 1 0]

top-middle-square: ['x' 'o' 'b'] -> [2 1 0]

top-right-square: ['x' 'o' 'b'] -> [2 1 0]

Class: ['positive' 'negative'] -> [1 0]

Processed dataset shape: torch.Size([958, 10])
Number of features: 9
Features: ['top-left-square', 'top-middle-square', 'top-right-square', 'middle-left-square', 'middle-middle-square', 'middle-right-square', 'bottom-left-square', 'bottom-middle-square', 'bottom-right-square']
Target: Class
Framework: PYTORCH
Data type: <class 'torch.Tensor'>

DECISION TREE CONSTRUCTION DEMO
Total samples: 958
Training samples: 766
Testing samples: 192

Constructing decision tree using tra

In [ ]:
# Test with the Nursery dataset
!python test.py --ID PES_F_PES2UG23CS353_Lab3 --data Nursery.csv

Running tests with PYTORCH framework
 target column: 'class' (last column)
Original dataset info:
Shape: (12960, 9)
Columns: ['parents', 'has_nurs', 'form', 'children', 'housing', 'finance', 'social', 'health', 'class']

First few rows:

parents: ['usual' 'pretentious' 'great_pret'] -> [2 1 0]

has_nurs: ['proper' 'less_proper' 'improper' 'critical' 'very_crit'] -> [3 2 1 0 4]

form: ['complete' 'completed' 'incomplete' 'foster'] -> [0 1 3 2]

class: ['recommend' 'priority' 'not_recom' 'very_recom' 'spec_prior'] -> [2 1 0 4 3]

Processed dataset shape: torch.Size([12960, 9])
Number of features: 8
Features: ['parents', 'has_nurs', 'form', 'children', 'housing', 'finance', 'social', 'health']
Target: class
Framework: PYTORCH
Data type: <class 'torch.Tensor'>

DECISION TREE CONSTRUCTION DEMO
Total samples: 12960
Training samples: 10368
Testing samples: 2592

Constructing decision tree using training data...

🌳 Decision tree construction completed using PYTORCH!

📊 OVERALL PERFORMANCE METR

# Analysis and Insights

## Performance Comparison

### Accuracy: Overall classification accuracy



* mushrooms.csv = 1.0000 (100.00%)
* tictactoe.csv = 0.8936 (89.36%)
* Nursery.csv = 0.9867 (98.67%)




### Precision: True positives / (True positives + False positives)

**Weighted**
* mushrooms.csv = 1.0000
* tictactoe.csv = 0.8930
* Nursery.csv = 0.9876

**Macro**
* mushrooms.csv = 1.0000
* tictactoe.csv = 0.8846
* Nursery.csv = 0.7604


### Recall: True positives / (True positives + False negatives)

**Weighted**
* mushrooms.csv = 1.0000
* tictactoe.csv = 0.8936
* Nursery.csv = 0.9867

**Macro**
* mushrooms.csv = 1.0000
* tictactoe.csv = 0.8846
* Nursery.csv = 0.7654


### F1-Score: Harmonic mean of precision and recall

**Weighted**
* mushrooms.csv = 1.0000
* tictactoe.csv = 0.8932
* Nursery.csv = 0.9872

**Macro**
* mushrooms.csv = 1.0000
* tictactoe.csv = 0.8816
* Nursery.csv = 0.7628


### **Insights**


*   Mushrooms dataset is perfectly classified
*   Nursery also performs extremely well in weighted metrics but macro metrics drop which implies that some classes are harder to predict for than others.
*  Tic-Tac-Toe has the lowest performance which implies the patterns are harder to generalize.



## Tree Characteristics Analysis

###  Tree Depth: Maximum depth of the constructed trees


* mushrooms.csv = 4
* tictactoe.csv = 7
* Nursery.csv = 7


### Number of Nodes: Total nodes in each tree

* mushrooms.csv = 59
* tictactoe.csv = 306
* Nursery.csv = 992

### Most Important Features: Attributes selected as root and early splits

* mushrooms.csv = 46
* tictactoe.csv = 196
* Nursery.csv = 710

### Tree Complexity: Relationship between tree size and dataset characteristics

* mushrooms.csv = 13
* tictactoe.csv = 110
* Nursery.csv = 282

### **Insights**


*   Mushrooms: It is a very Simple and Shallow tree.

*  Tic-Tac-Toe: It is a Deeper Tree with a larger size needed to capture the game logic.

*  Nursery: It is a very large tree and naturally the  complexity increases with this dataset.



## Dataset-Specific Insights

### Mushrooms

* Feature Importance: (Odor, spore-print-color, bruises) dominate in comparison to other columns.

* Class Distribution: Balanced.

* Decision Patterns: Early splits around odor.

* Overfitting Indicators: None since its a simple tree with perfect accuracy

### Tic-Tac-Toe

* Feature Importance: Middle squares (middle-middle-square) dominate since central positions are most decisive.

* Class Distribution: Balanced.

* Decision Patterns: Requires multiple splits to capture win conditions.

* Overfitting Indicators: Moderate tree depth; generalizes reasonably.

### Nursery

* Feature Importance:(Parents, has_nurs, form) are critical splits.

* Class Distribution: Imbalanced -inferred from differing macro and weighted average values

* Decision Patterns: Complex, with many deep splits.

* Overfitting Indicators: overfitting minority classes.

## Comparative Analysis Report

### Algorithm Performance



*   Highest Accuracy-Mushrooms because certain features directly map to desired output
*   Nursery is the Largest dataset and it also gives us the most complex decision tree. So I conclude that dataset size is proportional to complexity(i.e larger the dataset more complex the decision tree).
* More features imply more nodes



### Data Characteristics Impact

* We see that Nursery datset has a high class imbalance as inferred from the differing macro and weighted values of the dataset.It leads to more nodes being needed and an increased complexity.
* Binary features as seen in Mushroom datatset lead to cleaner splits and since it gives us the most accuracy amongst all the dataset we infer that Binary is better than multi-values features.
* Shallow trees are more interpretable than larger trees.

### Practical Applications




*   Mushrooms datset could be used in medical,food safety applications
*   Tic Tac Toe dataset could be used in reinforcement learning examples
* Nursery dataset could be used in social policy and recommendation systems.


